In [107]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
import json
import requests

In [ ]:
#to install similar packages do pip install -r requirements.txt

In [108]:
# create pinecone index
# pinecone index is one seperate db  inside your pinecone index

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
# if using openai then dimension = 1536a
pc.create_index(
  name="rag",
  dimension=1536,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
    ) 
)

#code above hosts it on aws using pinecone?

In [109]:
data = json.load(open("reviews.json"))


In [113]:
# Initialize OpenRouter client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Process review data and generate embeddings
processed_data = []
# Generate embeddings for each review
for review in data['reviews']:

  response = client.embeddings.create(
      input=review['review'],
      model="text-embedding-3-small"
  )
  embedding = response.data[0].embedding
        
  # Append the processed data
  processed_data.append({
      "values": embedding,
      "id": review["professor"],
      "metadata": {
          "review": review["review"],
          "subject": review["subject"],
          "stars": review["stars"]
      }
  })

In [ ]:
processed_data[0]

In [115]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [116]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}